In [21]:
import os
import cv2
import numpy as np

base_dir = "D:/6610210409/dataset/dataset"

def load_image_from_folder(folder):
    image = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            image.append(img)
    return images

In [22]:
base_dir = "D:/6610210409/dataset/dataset"
apple_images = load_images_from_folder(os.path.join(base_dir, "apple"))
orange_images = load_images_from_folder(os.path.join(base_dir, "orange"))

print(f"Loaded {len(apple_images)} apple images")
print(f"Loaded {len(orange_images)} orange images")

Loaded 50 apple images
Loaded 49 orange images


In [23]:
# you might want to resize or preprocess these image later
# Example : resize to a common size (e.g., 100x100)
resizeed_apple_images = [cv2.resize(img, (100, 100)) for img in apple_images]
resizeed_orange_images = [cv2.resize(img, (100, 100)) for img in orange_images]

In [25]:
#flatten the resized images into feature vactors
apple_features = [img.flatten() for img in resizeed_apple_images]
orange_features = [img.flatten() for img in resizeed_orange_images]

print(f"Flatten apple feature shape (first image): {apple_features[0].shape}")
print(f"Flatten orange feature shape (first image): {orange_features[0].shape}")

Flatten apple feature shape (first image): (30000,)
Flatten orange feature shape (first image): (30000,)


In [30]:
from sklearn.model_selection import train_test_split

# Create lables : 0 for apple, 1 foe apple
apple_labels = [0] * len(apple_features)
orange_labels = [1] * len(orange_features)

# combine feature and labels
all_features = apple_features + orange_features
all_labels = apple_labels + orange_labels

# Convert to nampy arrays
X = np.array(all_features)
y = np.array(all_labels)

# Shlit data into traning and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training features shape: {X_train.shape}")
print(f"Testing features shape: {X_test.shape}")
print(f"Training labels shape: {y_train.shape}")
print(f"Testing labels shape: {y_test.shape}")

Training features shape: (79, 30000)
Testing features shape: (20, 30000)
Training labels shape: (79,)
Testing labels shape: (20,)


In [31]:
from sklearn import svm
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the SVM classifier
svm_model = svm.SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# Preict on the test set
y_pred = svm_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Accuracy: 0.90

Classification Report:
              precision    recall  f1-score   support

           0       0.92      0.92      0.92        12
           1       0.88      0.88      0.88         8

    accuracy                           0.90        20
   macro avg       0.90      0.90      0.90        20
weighted avg       0.90      0.90      0.90        20



In [32]:
import joblib

# Save the trained model to a fole
filename = 'svm_image_classifier_model.pkl'
joblib.dump(svm_model, filename)

print(f"Model saved to {filename}")


Model saved to svm_image_classifier_model.pkl


In [41]:
%%writefile app.py

import streamlit as st
import joblib
import pickle
from PIL import Image
import numpy as np


# --- ---
with open("svm_image_classifier_model.pkl", "rb") as f:
model = joblib.load(f)

# --- UI ---
st.title("Fruit Classifier")
st.write(" ")

uploaded_file = st.file_uploader("Choose an image...", type=["jpg", "png", "jpeg"])
class_dict = {0: "Orange", 1: "Apple"}
if uploaded_file is not None:
#
image = Image.open(uploaded_file)
st.image(image, caption='Uploaded Image', use_container_width=True)

# --- ---
#
image = image.resize((100, 100))
image_array = np.array(image)
#
image_array = image_array.flatten().reshape(1, -1)
# --- ---
prediction = model.predict(image_array)[0]
# Get prediction name
prediction_name = class_dict[prediction]
st.write(f"Prediction: **{prediction_name}**")



Overwriting app.py
